In [1]:
%reset -f
# Import functions from the module
from netcdf_tools import get_points, get_values, select_netCDf
import netCDF4 as nc

In [2]:
import pandas as pd

df = pd.read_csv('../data/trajectories.csv')

trajectories_df = pd.DataFrame(df)

trajectories_date = trajectories_df['date'].unique()


In [3]:

end_of_string = '_FNMOC_10m_Surface_Winds.nc'
data_path = '../data/wind/'
u_wind_name = 'wnd_ucmp_height_above_ground'
date_of_trajectory = trajectories_date[0]
lon, lat, u_wind = select_netCDf(data_path,end_of_string,date_of_trajectory,u_wind_name)

TypeError: select_netCDf() takes 3 positional arguments but 4 were given

In [ ]:
u_wind = u_wind[0,0,:,:]
print(lon.shape)
print(lat.shape)

In [ ]:
daily_trajectories = get_points(trajectories_df, date_of_trajectory)
daily_trajectories.head(2)

In [ ]:
u_wind_on_site = get_values(lon, lat, u_wind, u_wind_name, daily_trajectories)
print(u_wind_on_site.head(2))


In [ ]:
# Temporary not working

'''
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import numpy as np

u = u_wind[:,:]
v = v_wind[:,:]
magnitude = np.sqrt(u**2+v**2)

fig, axes=plt.subplots(figsize=(10,10))
axes.set_title('Wind Speed and Direction in the Pacific Ocean on {}'.format(date_of_trajectory))
axes.set_xlabel('Longitude')
axes.set_ylabel('Latitude')
axes.set_xlim([-78.5,-77.5])
axes.set_ylim([-12.5,-11.5])


m=Basemap(projection='mill',lat_ts=15,llcrnrlon=lon.min(), \
          urcrnrlon=lon.max(),llcrnrlat=lat.min(),urcrnrlat=lat.max(), \
          resolution='i')

m.fillcontinents(color='0.6')
m.drawcoastlines()
x,y=m(*np.meshgrid(lon,lat))
#===============================
uproj,vproj,xx,yy=m.transform_vector(u,v,lon,lat,20,30,returnxy=True,masked=True)
#===============================

m.pcolormesh(x,y,magnitude,cmap=plt.cm.Reds)
m.quiver(xx,yy,uproj,vproj)
m.drawparallels(np.arange(-18,-2,1),labels=[1,0,0,0],linewidth=0.7,fontsize=9)
m.drawmeridians(np.arange(-86,-70,1),labels=[0,0,0,1],linewidth=0.7,fontsize=9)
m.colorbar()

# PLot the trajectories
x_1,y_1=m(daily_trajectories['longitude'].values,daily_trajectories['latitude'].values)
m.scatter(x_1, y_1, marker='o',color='b',s=10)

plt.show()
'''